In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import json
from scipy.spatial.distance import pdist, squareform
from numpy.linalg import inv
import matplotlib.pyplot as plt
import math
import random
import pickle

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Supporting functions

In [8]:
def DSD_calculator(adjacency, walk_length, restart_p):
    """
    adjacency - adjacency matrix represented as a numpy array
                assumes graph is fully connected.
    walk_length - the length of random walks used to calculate DSD
                  if walk_length = -1, then calculate DSD at convergence
    restart_p - the restart probability
        if p = 0, then it's a traditional random walk
    returns DSD matrix represented as a numpy array
    """
    # Normalization of the Adjacency Matrix
    adjacency = np.asmatrix(adjacency)
    n = adjacency.shape[0]
    degree = adjacency.sum(axis=1)
    p = adjacency / degree
    
    # random walk
    if walk_length >= 0:
        c = np.eye(n)
        for i in range(walk_length):
            c = (1 - restart_p) * np.dot(c, p) + restart_p * np.eye(n)
        return squareform(pdist(c,metric='cityblock'))
    else:
        pi = degree / degree.sum()
        return squareform(pdist(inv(np.eye(n) - p - pi.T),metric='cityblock'))

# Graph Construction

In [3]:
edges_df = pd.read_csv("data/git_web_ml/musae_git_edges.csv")
nodes_df = pd.read_csv("data/git_web_ml/musae_git_target.csv")

In [4]:
# G_github = nx.from_pandas_edgelist(edges_df, source="id_1", target='id_2')
# print('-----------------------')
# print('Original Graph:')
# print(G_github)
# # print(nx.info(G_github))
# pickle.dump(G_github,open('graph.pickleall','wb'))

# sample_num = 10000
# sample_nodes = random.choices(list(G_github.nodes()), k=sample_num)
# G = G_github.subgraph(sample_nodes)
# print('-----------------------')
# print('Picked graph with 3000 nodes:')
# print(G)
# # print(nx.info(G))
# largest_cc = max(nx.connected_components(G), key=len)
# G = G.subgraph(largest_cc)
# print('-----------------------')
# print('Fully Connected smaller graph:')
# print(G)
# # print(nx.info(G))
# pickle.dump(G,open('graph.pickle10k','wb'))

In [5]:
G = pickle.load(open('graph.pickle10k', 'rb'))
print(G)
print(nx.density(G))

Graph with 6123 nodes and 16591 edges
0.0008852072746100134


# Majority Voting

In [6]:
# traditional majority voting algorithm
positive_vote = 0
for i in list(G.nodes()):
    vote_0 = 0
    vote_1 = 0
    final_vote = 0

    for n in G.neighbors(i):
        label = nodes_df['ml_target'][n]
        weight = G.degree(n)

        if weight == 0:
            vote = 0
        else:
            vote = 1.0 / weight


        if label == 0:
            vote_0 = vote_0 + vote
        else:
            vote_1 = vote_1 + vote

    if vote_0 > vote_1:
        final_vote = 0
    else: 
        final_vote = 1

    if nodes_df['ml_target'][i] == final_vote:
        positive_vote = positive_vote + 1

print("accuracy = ", positive_vote / len(list(G.nodes())))   

accuracy =  0.8188796341662584


In [15]:
# # Calculate DSD
# edges = nx.to_numpy_array(G)
# # diameter = nx.diameter(G)
# # print("walk length is", 5 * diameter)
# DSD = DSD_calculator(edges, -1, 0)
# np.save("DSD.npy", DSD)

walk length is 50


In [31]:
DSD = np.load("DSD.npy")
print(DSD)

[[ 0.         14.46746307 15.53028767 ... 14.26510445 14.649793
  12.27262988]
 [14.46746307  0.         12.36110381 ... 17.5975529  18.61499113
  12.10867964]
 [15.53028767 12.36110381  0.         ... 17.36199121 18.26076509
  12.88720028]
 ...
 [14.26510445 17.5975529  17.36199121 ...  0.         12.28892266
  14.88334338]
 [14.649793   18.61499113 18.26076509 ... 12.28892266  0.
  15.39131094]
 [12.27262988 12.10867964 12.88720028 ... 14.88334338 15.39131094
   0.        ]]


In [35]:
# majority voting based on DSD 
positive_vote = 0
rows = len(DSD)

for i in range(rows):
    index_i = list(G.nodes())[i]
    vote_0 = 0
    vote_1 = 0
    degree_i = G.degree(index_i)
    
    t = G.degree(index_i)
    
    index_j_list = sorted(range(len(DSD[i])), key=lambda n: DSD[i][n])[:t]
    
    DSD_normalized = DSD[i][index_j_list]
    
    for j in range(len(index_j_list)):
        index_j = list(G.nodes())[j]
        label = nodes_df['ml_target'][index_j]
        degree_j = G.degree(index_j)
        
        if DSD_normalized[j] == 0:
            vote = 0
        else:
            vote = 1.0 / DSD_normalized[j]
        
        
        if label == 0:
            vote_0 = vote_0 + vote
        else:
            vote_1 = vote_1 + vote
            
    for n in G.neighbors(index_i):
        label = nodes_df['ml_target'][n]
        weight = G.degree(n)

        if weight == 0:
            vote = 0
        else:
            vote = 1.0 / weight


        if label == 0:
            vote_0 = vote_0 + vote
        else:
            vote_1 = vote_1 + vote
    
    if vote_0 > vote_1:
        final_vote = 0
    else:
        final_vote = 1
        
    if nodes_df['ml_target'][index_i] == final_vote:
        positive_vote = positive_vote + 1
    
print("accuracy = ", positive_vote / len(list(G.nodes())))

accuracy =  0.8272088845337253


In [36]:
# majority voting based on DSD 
positive_vote = 0
rows = len(DSD)

for i in range(rows):
    index_i = list(G.nodes())[i]
    vote_0 = 0
    vote_1 = 0
    degree_i = G.degree(index_i)
    
    t = G.degree(index_i)
    
    index_j_list = sorted(range(len(DSD[i])), key=lambda n: DSD[i][n])[:t]   
    DSD_normalized = DSD[i][index_j_list]
    
    for j in range(len(index_j_list)):
        index_j = list(G.nodes())[j]
        label = nodes_df['ml_target'][index_j]
        degree_j = G.degree(index_j)
        
        if DSD_normalized[j] == 0:
            vote = 0
        else:
            vote = 1.0 / DSD_normalized[j]
        
        
        if label == 0:
            vote_0 = vote_0 + vote
        else:
            vote_1 = vote_1 + vote
            
    
    if vote_0 > vote_1:
        final_vote = 0
    else:
        final_vote = 1
        
    if nodes_df['ml_target'][index_i] == final_vote:
        positive_vote = positive_vote + 1
    
print("accuracy = ", positive_vote / len(list(G.nodes())))

accuracy =  0.6413522782949534
